In [1]:
import os
import re
import pandas as pd
import numpy as np

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.plotly as py

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [12]:
custom_color_map=[
    "#9370DB", "#EE82EE", "#6495ED", "#DB7093", "#1E90FF", "#40E0D0", "#008B8B",
    "#2E8B57", "#00FF00", "#6B8E23", "#F0E68C", "#DAA520", "#FF8C00", "#FFA07A",
    "#FF7F50", "#DC143C", "#800000", "#D2691E", "#BC8F8F", "#5F9EA0", "#9ACD32",
    "#FFA500", "#FFD700", "#B22222", "#4B0082", "#00FFFF", "#808080", "#000080",
    "#000000", "#008080", "#FF0000", "#800000", "#FFF5EE", "#FFEFD5", "#5F9EA0",
]
default_color="#5F9EA0"

In [2]:
cwd=os.getcwd()
image_dir = os.path.join(cwd,'data/recipes_splitted/images')
image_dir_for_multimodal_classification = os.path.join(cwd, os.pardir,
                                                       "multimodal_classification","data")

df_extracted_features = pd.read_pickle(os.path.join(image_dir,"extracted_features.pkl"))

In [3]:
features=df_extracted_features.features.values.tolist()
labels=df_extracted_features.labels.values.tolist()
image_names=df_extracted_features.image_names.values.tolist()
index=df_extracted_features.index.values.tolist()

len_of_feature_vector=len(features[0])

unique_labels=sorted(set(labels))
number_of_colors=len(unique_labels)
possible_color_indicies=np.linspace(0,number_of_colors-1,number_of_colors,dtype=int)
labels2color_indices=dict(zip(unique_labels,possible_color_indicies))

print(labels2color_indices)

color_indicies=[]
for l in labels:
    color_index=labels2color_indices[l]
    color_indicies.append(color_index)

{'nachos': 17, 'french_toast': 8, 'sushi': 23, 'garlic_bread': 11, 'cheesecake': 1, 'onion_rings': 18, 'chocolate_mousse': 5, 'pizza': 20, 'apple_pie': 0, 'chicken_curry': 2, 'tacos': 24, 'steak': 22, 'dumplings': 7, 'hamburger': 12, 'chocolate_cake': 4, 'sashimi': 21, 'lasagna': 14, 'macarons': 16, 'macaroni_and_cheese': 15, 'frozen_yogurt': 10, 'tiramisu': 25, 'waffles': 26, 'fried_rice': 9, 'hummus': 13, 'deviled_eggs': 6, 'chicken_wings': 3, 'pancakes': 19}


In [4]:
df_extracted_features

,features,image_names,labels
0,"[0.05771321, 0.40469226, 0.17946579, 0.0, 0.0,...",1-1.jpg,apple_pie
1,"[0.24756677, 0.0, 0.81109595, 0.0, 0.0, 0.1121...",10-1.jpg,apple_pie
2,"[0.16576688, 0.0, 0.56071675, 0.0, 0.0, 0.0626...",11-1.jpg,apple_pie
3,"[0.0, 0.0, 0.3406139, 0.0, 0.0, 0.0, 0.4046788...",12-1.jpg,apple_pie
4,"[0.0, 0.09645958, 0.6799826, 0.14097536, 0.0, ...",13-1.jpg,apple_pie
5,"[0.0, 0.0, 0.9172021, 0.0, 0.0, 0.0, 0.0, 0.0,...",14-1.jpg,apple_pie
6,"[0.4587431, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",15-1.jpg,apple_pie
7,"[0.25609338, 0.64049345, 0.27605394, 0.0, 0.0,...",2-1.jpg,apple_pie
8,"[0.39920503, 0.26843926, 0.5381353, 0.3675958,...",3-1.jpg,apple_pie
9,"[0.49237722, 0.0, 0.0, 0.0, 0.0, 0.7390525, 0....",4-1.jpg,apple_pie


In [5]:
labels_for_mean_vectors=[]
names_for_mean_vectors=[]
mean_feature_vectors_of_images_in_recipes=[]
sum_of_vectors=[0]*len_of_feature_vector
num_of_images_in_recipe=0
last_image_name_prefix=0
current_image_name_prefix=1
i=0

while i < len(index):
    image_name=image_names[i]
    image_name_splitted=re.split('-',image_name)
    image_name_splitted=re.split('\.',image_name_splitted[0])
    last_image_name_prefix=current_image_name_prefix
    current_image_name_prefix=image_name_splitted[0]
    if int(current_image_name_prefix) == int(last_image_name_prefix):
        num_of_images_in_recipe=num_of_images_in_recipe+1
        sum_of_vectors=list(map(lambda x,y:x+y,sum_of_vectors,features[i]))
        i=i+1
    else:
        labels_for_mean_vectors.append(labels[i-1])
        names_for_mean_vectors.append(last_image_name_prefix)
        mean_feature_vector=list(map(lambda x:x/num_of_images_in_recipe,sum_of_vectors)) 
        mean_feature_vectors_of_images_in_recipes.append(mean_feature_vector)
        num_of_images_in_recipe=0
        sum_of_vectors=[0]*len_of_feature_vector
        
        
labels_for_mean_vectors.append(labels[i-1])
names_for_mean_vectors.append(last_image_name_prefix)
mean_feature_vector=list(map(lambda x:x/num_of_images_in_recipe,sum_of_vectors)) 
mean_feature_vectors_of_images_in_recipes.append(mean_feature_vector)

print(np.shape(mean_feature_vectors_of_images_in_recipes))

(316, 202)


In [6]:
#PCA

pca = PCA(n_components=2)
pca_2D_embeddings = pca.fit_transform(mean_feature_vectors_of_images_in_recipes)

dict_PCA_2D = {
    '1' : pca_2D_embeddings[:,0],
    '2' : pca_2D_embeddings[:,1],
    'mean_vector_labels' : labels_for_mean_vectors,
    'mean_vector_names': names_for_mean_vectors
}


df_PCA_2D=pd.DataFrame(dict_PCA_2D)
#print(df_2D_PCA)

In [15]:
traces=[]

for l in unique_labels:
    df_category=df_PCA_2D[(df_PCA_2D.mean_vector_labels==l)]
    point_name=str(df_category.mean_vector_labels[df_category.index[0]])
    color_index=labels2color_indices[l]
    color=custom_color_map[color_index] if color_index < len(custom_color_map) else default_color
    trace = go.Scatter(
    x=df_category['1'],
    y=df_category['2'],
    mode='markers',
    marker=dict(
        color=color,
        opacity=0.7,
        line=dict(color='rgb(140, 140, 170)')
    ),
    name=point_name,
    text=df_category.mean_vector_names,
    )

    traces.append(trace)

layout = go.Layout(
    title='PCA reduction: 2D features',
    hovermode='closest',
    margin=dict(
       l=30,
       r=30,
       b=30,
       t=30
    ),
      scene = dict(
        xaxis = dict(
            title='pca-1',
       ),
       yaxis = dict(
           title='pca-2',
       ),
    ),
 
)

fig = go.Figure(data=traces, layout=layout)
iplot(fig)

In [9]:
#t-SNE 2D
    
tsne2D = TSNE(perplexity=15, n_components=2, verbose=2, init='pca', n_iter=2000,
                  n_iter_without_progress=500, learning_rate=20,method='exact')
tsne_2D_embeddings = tsne2D.fit_transform(mean_feature_vectors_of_images_in_recipes)
    
dict_TSNE_2D = {
    '1' : tsne_2D_embeddings[:,0],
    '2' : tsne_2D_embeddings[:,1],
    'mean_vector_labels' : labels_for_mean_vectors,
    'mean_vector_names': names_for_mean_vectors
}

df_TSNE_2D=pd.DataFrame(dict_TSNE_2D)
#print(df_2D_PCA)

[t-SNE] Computing pairwise distances...
[t-SNE] Computed conditional probabilities for sample 316 / 316
[t-SNE] Mean sigma: 2.972833
[t-SNE] Iteration 50: error = 61.9941062, gradient norm = 0.1044406 (50 iterations in 0.332s)
[t-SNE] Iteration 100: error = 62.0087744, gradient norm = 0.1852614 (50 iterations in 0.299s)
[t-SNE] Iteration 150: error = 61.5167287, gradient norm = 0.1561795 (50 iterations in 0.297s)
[t-SNE] Iteration 200: error = 62.3326407, gradient norm = 0.1122405 (50 iterations in 0.350s)
[t-SNE] Iteration 250: error = 62.3452801, gradient norm = 0.0005444 (50 iterations in 0.296s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 62.345280
[t-SNE] Iteration 300: error = 0.9238836, gradient norm = 0.0026432 (50 iterations in 0.276s)
[t-SNE] Iteration 350: error = 0.8065307, gradient norm = 0.0008941 (50 iterations in 0.288s)
[t-SNE] Iteration 400: error = 0.7720754, gradient norm = 0.0005912 (50 iterations in 0.289s)
[t-SNE] Iteration 450: error = 0.

In [16]:
traces=[]

for l in unique_labels:
    df_category=df_TSNE_2D[(df_TSNE_2D.mean_vector_labels==l)]
    point_name=str(df_category.mean_vector_labels[df_category.index[0]])
    color_index=labels2color_indices[l]
    color=custom_color_map[color_index] if color_index < len(custom_color_map) else default_color
    trace = go.Scatter(
    x=df_category['1'],
    y=df_category['2'],
    mode='markers',
    marker=dict(
        color=color,
        opacity=0.7,
        line=dict(color='rgb(140, 140, 170)')
    ),
    name=point_name,
    text=df_category.mean_vector_names,
    )

    traces.append(trace)

layout = go.Layout(
    title='t-SNE reduction: 2D features',
    hovermode='closest',
    margin=dict(
       l=30,
       r=30,
       b=30,
       t=30
    ),
      scene = dict(
        xaxis = dict(
            title='pca-1',
       ),
       yaxis = dict(
           title='pca-2',
       ),
    ),
 
)

fig = go.Figure(data=traces, layout=layout)
iplot(fig)

In [24]:
#t-SNE 3D
    
tsne3D = TSNE(perplexity=15, n_components=3, verbose=2, init='pca',
                  n_iter=4000, n_iter_without_progress=1000, learning_rate=25, method='exact')
tsne_3D_embeddings = tsne3D.fit_transform(mean_feature_vectors_of_images_in_recipes)
    
dict_TSNE_3D = {
    '1' : tsne_3D_embeddings[:,0],
    '2' : tsne_3D_embeddings[:,1],
    '3' : tsne_3D_embeddings[:,2],
    'mean_vector_labels' : labels_for_mean_vectors,
    'mean_vector_names': names_for_mean_vectors
}

df_TSNE_3D=pd.DataFrame(dict_TSNE_3D)
#print(df_2D_PCA)

[t-SNE] Computing pairwise distances...
[t-SNE] Computed conditional probabilities for sample 316 / 316
[t-SNE] Mean sigma: 2.972833
[t-SNE] Iteration 50: error = 64.8885917, gradient norm = 0.0924183 (50 iterations in 0.529s)
[t-SNE] Iteration 100: error = 66.5673456, gradient norm = 0.0412105 (50 iterations in 0.560s)
[t-SNE] Iteration 150: error = 67.2187786, gradient norm = 0.0941736 (50 iterations in 0.513s)
[t-SNE] Iteration 200: error = 66.7584484, gradient norm = 0.0367500 (50 iterations in 0.485s)
[t-SNE] Iteration 250: error = 66.1734178, gradient norm = 0.0303886 (50 iterations in 0.515s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 66.173418
[t-SNE] Iteration 300: error = 0.7926945, gradient norm = 0.0026847 (50 iterations in 0.515s)
[t-SNE] Iteration 350: error = 0.7088396, gradient norm = 0.0003890 (50 iterations in 0.499s)
[t-SNE] Iteration 400: error = 0.6652789, gradient norm = 0.0007509 (50 iterations in 0.496s)
[t-SNE] Iteration 450: error = 0.

In [26]:
traces=[]

for l in unique_labels:
    df_category=df_TSNE_3D[(df_TSNE_3D.mean_vector_labels==l)]
    point_name=str(df_category.mean_vector_labels[df_category.index[0]])
    color_index=labels2color_indices[l]
    color=custom_color_map[color_index] if color_index < len(custom_color_map) else default_color
    trace = go.Scatter3d(
    x=df_category['1'],
    y=df_category['2'],
    z=df_category['3'],
    mode='markers',
    marker=dict(
        color=color,
        opacity=0.7,
        line=dict(color='rgb(140, 140, 170)')
    ),
    name=point_name,
    text=df_category.mean_vector_names,
    )

    traces.append(trace)

layout = go.Layout(
    title='t-SNE reduction: 3D features',
    hovermode='closest',
    margin=dict(
       l=30,
       r=30,
       b=30,
       t=30
    ),
      scene = dict(
        xaxis = dict(
            title='x',
       ),
       yaxis = dict(
           title='y',
       ),
       zaxis = dict(
           title='z',
       ),
    ),
 
)

fig = go.Figure(data=traces, layout=layout)
iplot(fig)

In [11]:
dict_mean_feature_vectors = {
    'mean_feature_vectors': mean_feature_vectors_of_images_in_recipes,
    'mean_vector_labels' : labels_for_mean_vectors,
    'mean_vector_names': names_for_mean_vectors
}

df_mean_feature_vectors=pd.DataFrame(dict_mean_feature_vectors)


print("Saving mean feature vectors to: ",
      os.path.join(image_dir,"mean_feature_vectors.pkl"))
df_mean_feature_vectors.to_pickle(os.path.join(image_dir,"mean_feature_vectors.pkl"))

print("Saving mean feature vectors for multimodal classification to: ",
      os.path.join(image_dir_for_multimodal_classification,"imgs_mean_feature_vectors.pkl"))
df_mean_feature_vectors.to_pickle(os.path.join(image_dir_for_multimodal_classification, "imgs_mean_feature_vectors.pkl"))


print("Mean feature vectors pandas dataframe: \n")
df_mean_feature_vectors

Saving mean feature vectors to:  /notebooks/school/image_feature_extraction/data/recipes_splitted/images/mean_feature_vectors.pkl
Mean feature vectors pandas dataframe: 



,mean_feature_vectors,mean_vector_labels,mean_vector_names
0,"[0.057713210582733154, 0.40469226241111755, 0....",apple_pie,1
1,"[0.24756677448749542, 0.0, 0.8110959529876709,...",apple_pie,10
2,"[0.1657668799161911, 0.0, 0.5607167482376099, ...",apple_pie,11
3,"[0.0, 0.0, 0.3406139016151428, 0.0, 0.0, 0.0, ...",apple_pie,12
4,"[0.0, 0.09645958244800568, 0.679982602596283, ...",apple_pie,13
5,"[0.0, 0.0, 0.9172021150588989, 0.0, 0.0, 0.0, ...",apple_pie,14
6,"[0.4587430953979492, 0.0, 0.0, 0.0, 0.0, 0.0, ...",apple_pie,15
7,"[0.2560933828353882, 0.6404934525489807, 0.276...",apple_pie,2
8,"[0.3992050290107727, 0.26843926310539246, 0.53...",apple_pie,3
9,"[0.49237722158432007, 0.0, 0.0, 0.0, 0.0, 0.73...",apple_pie,4
